In [ ]:
""" %%bash
# create virtual environment, activate it and install packages. If you already have a venv (from previous assignment for example) or
# you're not even using one, you can skip this step.
python3 -m venv .venv
source .venv/bin/activate
pip install numpy matplotlib keras tensorflow[and-cuda] """

  Using cached numpy-2.1.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.6 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.datasets import cifar10
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization
from keras.models import Sequential
from keras.utils import to_categorical
from keras.regularizers import l1, l2

2024-11-30 16:53:52.263061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732982032.486782    9761 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732982032.549460    9761 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 16:53:53.238757: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
#CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
print(train_images.shape)
train_images = train_images.reshape(train_images.shape[0], 32, 32, 3) #ensure shape 32 W x 32 H x 3 channels for each image
test_images = test_images.reshape(test_images.shape[0], 32, 32, 3)

#range 0-1
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

#One-hot encoding labels
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filepath='best_weights.h5',
    monitor='val_accuracy',              
    save_best_only=True,                 
    mode='max',  # Mode for monitoring, 'max' for accuracy
    verbose=1                        
)

#1st try LeNet-5 with batch normalization and l2 regularization - about 0.66 test validation with 0.0001 lambda and 0.001 le, I believe
"""
model = Sequential()
l2_lambda = 0.0001
model.add(Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(32, 32, 3), kernel_regularizer=l2(l2_lambda))) #C1
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) #S1
model.add(Conv2D(16, kernel_size=(5, 5), activation='relu', kernel_regularizer=l2(l2_lambda))) #C2
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2))) #S2
model.add(Flatten())
model.add(Dense(120, activation='relu', kernel_regularizer=l2(l2_lambda)))  # FC1
model.add(Dense(84, activation='relu', kernel_regularizer=l2(l2_lambda)))  # FC2
model.add(Dense(10, activation='softmax', kernel_regularizer=l2(l2_lambda)))  # FC3
#model.summary()
 """

#currently trying ResNet50
model = Sequential()
model = keras.applications.ResNet50(include_top=True,
                                    input_shape=(32, 32, 3),
                                    pooling='avg',
                                    weights=None,
                                    classes=10)

model.compile(loss=keras.metrics.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.002),
              metrics=['accuracy'])

output = model.fit(train_images,
           train_labels,
           batch_size=128,
           epochs=100,
           verbose=1,
           validation_data=(test_images, test_labels),
           callbacks=[checkpoint_callback])

model.save('model.h5')

ModuleNotFoundError: No module named 'tensorflow'